So We first Initialize our maze with random walls. our maze size is Dynamic. We can also use a file to read our maze.

In [11]:

# this script is written to create a bunch of maze maps for evaluating the search algorithms:
import numpy as np
import matplotlib.pyplot as  plot



In [12]:
# embedding :
empty_cell = 0
wall = 1
start_cell = 2
goal_cell = 3
# constants:
width = 8# width of the map
height = 8 # height of the map
wall_ratio = 0.2 # ratio of walls in the map
cluster_attraction_weight = 6

#creating an empty map to start with:
map = np.full(shape=(height+2,width+2),fill_value=empty_cell)
hit_map = np.full(shape=(height+2,width+2),fill_value=1) # for deciding where to put the next wall
map[0] = wall
map[-1] = wall
hit_map[0] = 0
hit_map[-1] = 0
for i in range(1,height+1) :
    map[i][0] = wall
    map[i][-1] = wall
    hit_map[i][0] = 0
    hit_map[i][-1] = 0
num_walls = np.random.binomial(n=height*width,p=wall_ratio)
for i in range(num_walls):
    next_wall_loc = np.random.choice((height+2)*(width+2),p=hit_map.flatten()/sum(hit_map.flatten()))
    x_next_wall = next_wall_loc//(width+2)
    y_next_wall = next_wall_loc%(width+2)
    map[x_next_wall][y_next_wall] = wall
    hit_map[x_next_wall][y_next_wall] = 0

    if hit_map[x_next_wall-1][y_next_wall] == 1:
        hit_map[x_next_wall - 1][y_next_wall] = cluster_attraction_weight

    if hit_map[x_next_wall+1][y_next_wall] == 1:
        hit_map[x_next_wall + 1][y_next_wall] = cluster_attraction_weight

    if hit_map[x_next_wall][y_next_wall-1] == 1:
        hit_map[x_next_wall][y_next_wall-1] = cluster_attraction_weight

    if hit_map[x_next_wall][y_next_wall+1] == 1:
        hit_map[x_next_wall][y_next_wall+1] = cluster_attraction_weight

# now we only need to add start and goal cells:
empty_cells = (hit_map > 0).flatten()
empty_cells =empty_cells / sum(empty_cells)
start_loc,goal_loc = np.random.choice((height + 2) * (width + 2), p=empty_cells,size=2,replace=False)
x_start = start_loc // (width + 2)
y_start = start_loc % (width + 2)
x_goal = goal_loc // (width + 2)
y_goal = goal_loc % (width + 2)

map[x_start][y_start] = start_cell
map[x_goal][y_goal] = goal_cell
# saving the map in numpy format:
np.save("random_map.npy",map)
# and also in a text file :
fp = open("random_map.txt",'w')
fp.write(np.array2string(map))
print(map)


[[1 1 1 1 1 1 1 1 1 1]
 [1 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 1 0 1]
 [1 0 0 3 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 1 0 0 0 1]
 [1 0 0 0 0 1 1 0 1 1]
 [1 1 1 1 2 0 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]


if We want to initalize our maze with an existing file we will use this cell

In [13]:
map = np.load('random_map.npy')
print(map)

[[1 1 1 1 1 1 1 1 1 1]
 [1 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 1 0 1]
 [1 0 0 3 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 1 0 0 0 1]
 [1 0 0 0 0 1 1 0 1 1]
 [1 1 1 1 2 0 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]


So we now that we had initialize our maze we go for finding a path with dfs. we will also write how many nodes we actually visited to reach this goal.



In [14]:
def valid(x):
    if x == 1 or x==8:
        return False
    else:
        return True
def isGoal(x):
    if x == 3:
        return True
    else:
        return False


In [15]:
m = np.copy(map)
l = []
flag = True
def DFS(x,y):
    global flag
    l.append({x,y})
    if m[x][y]==3:
        flag = False
        return
    m[x][y] = 8
 
    if valid(m[x][y+1]) and flag:
        DFS(x,y+1)
    
    if valid(m[x+1][y])and flag:
        DFS(x+1,y)
    
    if valid(m[x][y-1])and flag:
        DFS(x,y-1)
    
    if valid(m[x-1][y])and flag:
        DFS(x-1,y)
    
    return
DFS(x_start,y_start)
m[x_start][y_start]=2
print(m)
print("Path is: ",l)
print("Number of Nodes Visited = ",len(l))

[[1 1 1 1 1 1 1 1 1 1]
 [1 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 1 0 1]
 [1 0 8 3 0 0 0 0 0 1]
 [1 8 8 8 8 8 8 8 8 1]
 [1 8 8 8 8 8 8 8 8 1]
 [1 8 8 8 8 1 8 8 8 1]
 [1 8 8 8 8 1 1 8 1 1]
 [1 1 1 1 2 8 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
Path is:  [{8, 4}, {8, 5}, {4, 7}, {3, 7}, {2, 7}, {1, 7}, {1, 6}, {2, 6}, {3, 6}, {4, 6}, {4, 5}, {5}, {5, 6}, {5, 7}, {8, 5}, {8, 6}, {6, 7}, {7}, {6}, {8, 4}, {4, 7}, {4, 6}, {4, 5}, {4}, {3, 4}, {3, 5}, {2, 5}, {1, 5}, {1, 4}, {2, 4}, {2, 3}, {3}]
Number of Nodes Visited =  32


So this the Maze Solving using BFS 

In [16]:
bm = np.copy(map)
l = []
q = []
flag = True
def BFS_Check(x,y):
    global bm
    global flag
 
    if valid(bm[x][y+1]) and flag:
        l.append({x,y+1})        
        if isGoal(bm[x][y+1]):
            flag = False
            return
        else:
            q.append([x,y+1])
            bm[x][y+1]=8
    
    if valid(bm[x+1][y])and flag:
        l.append({x+1,y})
        if isGoal(bm[x+1][y]):
            flag = False
            return
        else:
            q.append([x+1,y])
            bm[x+1][y]=8
    
    if valid(bm[x][y-1])and flag:
        l.append({x,y-1})        
        if isGoal(bm[x][y-1]):
            flag = False
            return
        else:
            q.append([x,y-1])
            bm[x][y-1]=8
    
    if valid(bm[x-1][y])and flag:
        l.append({x-1,y})
        if isGoal(bm[x-1][y]):
            flag = False
            return
        else:
            q.append([x-1,y])
            bm[x-1][y]=8
    
    return
def BFS(x,y):
    global flag
    q.append([x,y])
    l.append({x,y})
    bm[x][y]=0
    while len(q) != 0 and flag:
        t = q.pop(0)
        BFS_Check(t[0],t[1])
        
BFS(x_start,y_start)
bm[x_start][y_start]=2
print(bm)

print("Path is: ",l)
print("Number of Nodes Visited = ",len(l))

[[1 1 1 1 1 1 1 1 1 1]
 [1 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 1 0 1]
 [1 0 0 3 8 0 0 0 0 1]
 [1 0 8 8 8 8 0 0 0 1]
 [1 8 8 8 8 8 8 0 0 1]
 [1 8 8 8 8 1 0 0 0 1]
 [1 8 8 8 8 1 1 0 1 1]
 [1 1 1 1 2 8 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
Path is:  [{8, 4}, {8, 5}, {4, 7}, {8, 4}, {3, 7}, {4, 6}, {2, 7}, {3, 6}, {4, 5}, {1, 7}, {2, 6}, {3, 5}, {5}, {4}, {1, 6}, {2, 5}, {3, 4}, {5, 6}, {4, 5}, {3, 4}, {1, 5}, {2, 4}, {3}]
Number of Nodes Visited =  23


We also do the search with IDS the only difference for IDS and DFS is a depth limit

In [17]:
m = np.copy(map)
l = []
flag = True
depth_limit = 6
def IDFS(x,y,d):
    l.append({x,y})
    global flag
    global depth_limit
    if m[x][y]==3:
        flag = False
        return
    m[x][y] = 8
    if d==depth_limit:
        return
    if valid(m[x][y+1]) and flag:
        IDFS(x,y+1,d+1)
    
    if valid(m[x+1][y])and flag:
        IDFS(x+1,y,d+1)
    
    if valid(m[x][y-1])and flag:
        IDFS(x,y-1,d+1)
    
    if valid(m[x-1][y])and flag:
        IDFS(x-1,y,d+1)
    
    return
IDFS(x_start,y_start,0)
m[x_start][y_start]=2
print(m)
print("Path is: ",l)
print("Number of Nodes Visited = ",len(l))

[[1 1 1 1 1 1 1 1 1 1]
 [1 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 1 0 1]
 [1 0 0 3 0 0 0 0 0 1]
 [1 0 0 0 8 0 0 0 0 1]
 [1 8 0 8 8 8 0 0 0 1]
 [1 8 8 8 8 1 0 0 0 1]
 [1 8 8 8 8 1 1 0 1 1]
 [1 1 1 1 2 8 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
Path is:  [{8, 4}, {8, 5}, {4, 7}, {3, 7}, {2, 7}, {1, 7}, {1, 6}, {2, 6}, {1, 5}, {3, 6}, {4, 6}, {4, 5}, {5}, {3, 5}, {4}]
Number of Nodes Visited =  15


Now we go for A*
Note : For heuristic function we will calculate the distance our location to goal location. ((x_goal-x)^2 + (y_goal-y)^2))^0.5 so we always want to go straight to the goal.

In [18]:
import math
fmap = np.full(shape=(height+2,width+2),fill_value=10000)
am = np.copy(map)
l = []
q = []
flag = True

def minValue(b):
    min = 10000
    for i in b:
        if fmap[i[0],i[1]] <= min:
            min = fmap[i[0],i[1]]
            ind = b.index(i)
    return ind

def h(x,y):
    a = (x_goal - x) * (x_goal - x)
    b = (y_goal - y) * (y_goal - y)
    return math.sqrt(a+b)

def aStar_Check(x,y,cost):
    global flag
 
    if valid(am[x][y+1]) and flag:
        l.append({x,y+1})
        g = cost + h(x,y+1)
        fmap[x][y+1] = min(fmap[x][y+1], g)
        if isGoal(am[x][y+1]):
            flag = False
            return
        else:
            q.append([x,y+1])
            am[x][y+1]=8
    
    if valid(am[x+1][y])and flag:
        l.append({x+1,y})        
        g = cost + h(x+1,y)
        fmap[x+1][y] = min(fmap[x+1][y], g)
        if isGoal(am[x+1][y]):
            flag = False
            return
        else:
            q.append([x+1,y])
            am[x+1][y]=8
    
    if valid(am[x][y-1])and flag:
        l.append({x,y-1})     
        g = cost + h(x,y-1)
        fmap[x][y-1] = min(fmap[x][y-1], g)        
        if isGoal(am[x][y-1]):
            flag = False
            return
        else:
            q.append([x,y-1])
            am[x][y-1]=8
    
    if valid(am[x-1][y])and flag:
        l.append({x-1,y})     
        g = cost + h(x-1,y)
        fmap[x-1][y] = min(fmap[x-1][y], g)   
        if isGoal(am[x-1][y]):
            flag = False
            return
        else:
            q.append([x-1,y])
            am[x-1][y]=8
    
    return
def aStar(x,y):
    global flag
    q.append([x,y])
    l.append({x,y})
    fmap[x][y] = h(x,y)
    am[x][y] = 8
    while len(q) != 0 and flag:
        t = q.pop(minValue(q))
        aStar_Check(t[0],t[1],fmap[t[0]][t[1]])
        
aStar(x_start,y_start)
am[x_start][y_start]=2
print(am)
print(fmap)

print("Path is: ",l)
print("Number of Nodes Visited = ",len(l))

[[1 1 1 1 1 1 1 1 1 1]
 [1 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 1 0 1]
 [1 0 0 3 8 8 0 0 0 1]
 [1 0 0 8 8 8 0 0 0 1]
 [1 0 0 8 8 8 0 0 0 1]
 [1 0 8 8 8 1 0 0 0 1]
 [1 0 8 8 8 1 1 0 1 1]
 [1 1 1 1 2 8 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
[[10000 10000 10000 10000 10000 10000 10000 10000 10000 10000]
 [10000 10000 10000 10000 10000 10000 10000 10000 10000 10000]
 [10000 10000 10000 10000 10000 10000 10000 10000 10000 10000]
 [10000 10000 10000    16    16    18 10000 10000 10000 10000]
 [10000 10000 10000    16    15    17 10000 10000 10000 10000]
 [10000 10000 10000    16    14    16 10000 10000 10000 10000]
 [10000 10000    18    15    12 10000 10000 10000 10000 10000]
 [10000 10000    17    13     9 10000 10000 10000 10000 10000]
 [10000 10000 10000 10000     5    10 10000 10000 10000 10000]
 [10000 10000 10000 10000 10000 10000 10000 10000 10000 10000]]
Path is:  [{8, 4}, {8, 5}, {4, 7}, {3, 7}, {4, 6}, {3, 6}, {4, 5}, {2, 7}, {5}, {3, 5}, {4}, {4, 5}, {3, 4}, {3, 4}, {2, 6}, {3, 5}, {3}]


For RBFS:

In [20]:
rmap = np.full(shape=(height+2,width+2),fill_value=10000)
parent = []
parentH = []
rm = np.copy(map)
l = []
q = []
flag = True

def Rminvalue(b):
    min = 10001
    ind = 0
    for i in b:
        if rmap[i[0],i[1]] <= min:
            min = rmap[i[0],i[1]]
            ind = b.index(i)
    return ind

def secondMinValue(b):
    min = 10001
    ind = 0
    for i in b:
        if rmap[i[0],i[1]] <= min:
            lind = ind
            min = rmap[i[0],i[1]]
            ind = b.index(i)
    return lind

def ldif(first, second):
    diflist = []
    for i in first:
        if i not in second:
            diflist.append(i)
    return diflist

def RBFS_Check(x,y,cost):
    global flag
    global q
 
    if valid(rm[x][y+1]) and flag:
        l.append({x,y+1})
        g = cost + h(x,y+1)
        rmap[x][y+1] = min(rmap[x][y+1], g)
        if isGoal(rm[x][y+1]):
            flag = False
            return
        else:
            q.append([x,y+1])
            print("q after update: ", q)
            rm[x][y+1]=8
    
    if valid(rm[x+1][y]) and flag:
        l.append({x+1,y})        
        g = cost + h(x+1,y)
        rmap[x+1][y] = min(rmap[x+1][y], g)
        if isGoal(rm[x+1][y]):
            flag = False
            return
        else:
            q.append([x+1,y])
            print("q after update: ", q)
            rm[x+1][y]=8
    
    if valid(rm[x][y-1]) and flag:
        l.append({x,y-1})     
        g = cost + h(x,y-1)
        rmap[x][y-1] = min(rmap[x][y-1], g)        
        if isGoal(rm[x][y-1]):
            flag = False
            return
        else:
            q.append([x,y-1])
            print("q after update: ", q)
            rm[x][y-1]=8
    
    if valid(rm[x-1][y]) and flag:
        l.append({x-1,y})     
        g = cost + h(x-1,y)
        rmap[x-1][y] = min(rmap[x-1][y], g)   
        if isGoal(rm[x-1][y]):
            flag = False
            return
        else:
            q.append([x-1,y])
            print("q after update: ", q)
            rm[x-1][y]=8
    
    return

def RBFS(x,y):
    global q
    global flag
    q.append([x,y])
    l.append({x,y})
    rm[x][y] = 8
    rmap[x][y] = h(x,y)
    t = q.pop(Rminvalue(q))
    RBFS_Check(t[0],t[1],rmap[t[0]][t[1]])
    secondLeastF = q[secondMinValue(q)]
    print("min, secmin :",q[Rminvalue(q)],secondLeastF)
    #print("q is: ",q)
    #print("l is: ",l)
    print(rmap)
    print(rm)
    print("parents :", parent, parentH)
    while len(q) != 0 and flag:
        #print("q is: ",q)
        #print("l is: ",l)
        print(rmap)
        print(rm)
        print("parents :", parent, parentH)
        qBackup = q.copy()
        t = q.pop(Rminvalue(q))
        if t in parent:
            ind=parent.index(t)
            rmap[t[0]][t[1]]=parentH[ind]
            parent.pop(ind)
            parentH.pop(ind)
        RBFS_Check(t[0],t[1],rmap[t[0]][t[1]])
        print("min, secmin :",q[Rminvalue(q)],secondLeastF)
        if q[Rminvalue(q)]==secondLeastF:
            print("q is :", q)
            print("backup is:", qBackup)
            diff = ldif(q,qBackup)
            print("diff is: ",diff)
            if len(diff) > 0:
                newF=diff[Rminvalue(diff)]
                parent.append(t)
                parentH.append(rmap[t[0]][t[1]])
                rmap[t[0]][t[1]]=rmap[newF[0]][newF[1]]
                for j in diff:
                    rmap[j[0]][j[1]] = 10000
                    rm[j[0]][j[1]] = 0
                q = qBackup.copy()
        secondLeastF = q[secondMinValue(q)]
            
            
            
        
RBFS(x_start,y_start)
rm[x_start][y_start]=2
print(rm)
print(rmap)

print("Path is: ",l)
print("Number of Nodes Visited = ",len(l))

q after update:  [[8, 5]]
q after update:  [[8, 5], [7, 4]]
min, secmin : [7, 4] [8, 5]
[[10000 10000 10000 10000 10000 10000 10000 10000 10000 10000]
 [10000 10000 10000 10000 10000 10000 10000 10000 10000 10000]
 [10000 10000 10000 10000 10000 10000 10000 10000 10000 10000]
 [10000 10000 10000 10000 10000 10000 10000 10000 10000 10000]
 [10000 10000 10000 10000 10000 10000 10000 10000 10000 10000]
 [10000 10000 10000 10000 10000 10000 10000 10000 10000 10000]
 [10000 10000 10000 10000 10000 10000 10000 10000 10000 10000]
 [10000 10000 10000 10000     9 10000 10000 10000 10000 10000]
 [10000 10000 10000 10000     5    10 10000 10000 10000 10000]
 [10000 10000 10000 10000 10000 10000 10000 10000 10000 10000]]
[[1 1 1 1 1 1 1 1 1 1]
 [1 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 1 0 1]
 [1 0 0 3 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 1 0 0 0 1]
 [1 0 0 0 8 1 1 0 1 1]
 [1 1 1 1 8 8 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
parents : [] []
[[10000 10000 10000 10000 10000 10

In [88]:
j = [[2, 7], [1, 6]]
o = [[2, 7], [2, 6], [1, 5]]
print(ldif(o,j))

[[2, 6], [1, 5]]
